# Gabe, develop this script!
Alot of this is working in the other script but needs to be checked. The GPIO is really what needs work.

In [1]:
# Imports
import numpy as np
import scipy
import scipy.signal as signal
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.interpolate import CubicSpline
from scipy.signal import hilbert
import numpy as np
import ipywidgets as ipw
import base64
from random import randint
from pynq import Clocks
import xrfdc
import os
from pynq.lib import Pmod_IO
import time

# Use the RFSoC base overlay
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

# Start RF clocks
base.init_rf_clks()

# The below block doesn't need changes
Change sample_scaler to reduce/increase number_samples.
Change adc_array to change which ADCs you're using.

In [2]:
# ADC initialization
# Channels
DAC_CHANNEL_B = 0 # 'Channel 0': {'Tile': 224, 'Block': 0}
DAC_CHANNEL_A = 1 # 'Channel 1': {'Tile': 230, 'Block': 0}

ADC_CHANNEL_D = 0 # 'Channel 0': {'Tile': 224, 'Block': 0}
ADC_CHANNEL_C = 1 # 'Channel 1': {'Tile': 224, 'Block': 1}
ADC_CHANNEL_B = 2 # 'Channel 2': {'Tile': 226, 'Block': 0}
ADC_CHANNEL_A = 3 # 'Channel 3': {'Tile': 226, 'Block': 1}

adc_array = [ADC_CHANNEL_C, ADC_CHANNEL_A]

adc_char_array = ['D', 'C', 'B', 'A']

sample_scaler = 2048
number_samples = int(32768/sample_scaler)  # Between 16 and 32768
decimation_factor = 1 # 2 is default
sample_frequency = 4915.2e6/decimation_factor  # Hz The default sample frequency is 4915.2e6 Hz which is sufficient for our signal

original_adc_settings = base.radio.receiver.channel[ADC_CHANNEL_D].adc_block.MixerSettings

for ADC in adc_array:
    base.radio.receiver.channel[ADC].adc_block.DecimationFactor = decimation_factor
    base.radio.receiver.channel[ADC].adc_block.MixerSettings = {
        'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
        'EventSource':    xrfdc.EVNT_SRC_TILE, 
        'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
        'Freq':           0.0,
        'MixerMode':      xrfdc.MIXER_MODE_R2C,
        'MixerType':      xrfdc.MIXER_TYPE_FINE,
        'PhaseOffset':    0.0
    }
    base.radio.receiver.channel[ADC].adc_block.UpdateEvent(xrfdc.EVENT_MIXER)
    
print("Original ADC settings:", original_adc_settings)
print("New ADC settings:", base.radio.receiver.channel[ADC].adc_block.MixerSettings)

Original ADC settings: {'Freq': -1228.8, 'PhaseOffset': 0.0, 'EventSource': 2, 'CoarseMixFreq': 0, 'MixerMode': 3, 'FineMixerScale': 1, 'MixerType': 2}
New ADC settings: {'Freq': 0.0, 'PhaseOffset': 0.0, 'EventSource': 2, 'CoarseMixFreq': 0, 'MixerMode': 3, 'FineMixerScale': 1, 'MixerType': 2}


# Calculate beamforming weights
Use steering angle, careful attention needed when passing them into GPIO to handle -30 vs 30 degrees case.

In [3]:
d = 0.5 # half wavelength spacing
Nr = len(adc_array)
steering_angle_degrees = 30 
steering_angle = steering_angle_degrees / 180 * np.pi # convert to radians
beamforming_weights = np.exp(-2j * np.pi * d * np.arange(Nr) * np.sin(steering_angle)) # array factor
print("beamforming_weights:", beamforming_weights)

beamforming_weights: [1.00000000e+00+0.j 2.83276945e-16-1.j]


# Pass them into GPIO
Work closely with Katie on this. See this link: https://pynq.readthedocs.io/en/latest/pynq_libraries/psgpio.html

In [4]:
# Add GPIO Code here

# Transfer data from channel that Katie ports beamforming data to


In [10]:
beamformed_data = []
# Something like this:
beamformed_data.append(base.radio.receiver.channel[0].transfer(number_samples))

16


# Plot beamformed data
Also add code to calculate max power

In [7]:
beamformed_interpolated_data = []

# Create Plotly figure for interpolated ADC data
beamformed_fig = make_subplots(specs=[[{"secondary_y": False}]])  # Adjust as necessary

# Needed for interpolation

time_data = np.arange(0, number_samples/sample_frequency, 1/sample_frequency)
print(len(time_data))
print(len(beamformed_data))
dense_t = np.linspace(time_data.min(), time_data.max(), len(time_data) * 10)  # Increase density
    
sampled_signal = np.real(beamformed_data)
cs_real = CubicSpline(time_data, sampled_signal)

# Beamformed interpolated data
beamformed_interpolated_data.append(cs_real(dense_t))

# Add beamformed interpolated data trace
beamformed_fig.add_trace(
go.Scatter(x=dense_t, y=beamformed_interpolated_data[index], name=f"Beamformed Interpolated Data"),
secondary_y=False,
)

# Update layout of raw ADC data
beamformed_fig.update_layout(
title=f"Time Domain Plot of ADC Channels",
xaxis_title="Time(s)",
yaxis_title="Amplitude(V)",
)

# Show beamformed ADC data
beamformed_fig.show()  

16
1


ValueError: The length of `y` along `axis`=0 doesn't match the length of `x`

# Can also plot raw data by plotting data from a channel that isn't being used to port beamforming data
Also add code to calculate max power

In [ ]:
raw_data = []
raw_interpolated_data = []

# Create Plotly figure for interpolated ADC data
raw_fig = make_subplots(specs=[[{"secondary_y": False}]])  # Adjust as necessary

# Needed for interpolation
time_data = np.arange(0, number_samples/sample_frequency, 1/sample_frequency)
dense_t = np.linspace(time_data.min(), time_data.max(), len(time_data) * 10)  # Increase density
    
# Get chunk of data, interpolate it, and create trace
raw_data.append(base.radio.receiver.channel[ADC].transfer(number_samples))
sampled_signal = np.real(raw_data)
cs_real = CubicSpline(time_data, sampled_signal)

# Raw interpolated data
raw_interpolated_data.append(cs_real(dense_t))

# Add raw interpolated data trace
raw_fig.add_trace(
go.Scatter(x=dense_t, y=raw_interpolated_data, name=f"Interpolated Data ADC {adc_char_array[ADC]}"),
secondary_y=False,
)

# Update layout of raw ADC data
raw_fig.update_layout(
title=f"Time Domain Plot of ADC Channels",
xaxis_title="Time(s)",
yaxis_title="Amplitude(V)",
)

# Show raw ADC data
raw_fig.show() 